In [1]:
import pandas as pd

file = "MiddleEast_2015-2024_Nov01.csv"

df = pd.read_csv(file)
df


,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,PSE62716,2024-11-01,2024,1,Strategic developments,Strategic developments,Looting/property destruction,Settlers (Israel),NaN,Political militia,...,Shi'b Al Batim,31.3942,35.1365,1,PLO Negotiations Affairs Department,Other,"Land seizure: On 1 November 2024, Israeli sett...",0,NaN,1730758098
1,YEM93235,2024-11-01,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Yemen),Government of Yemen (2017-) Houthi,Protesters,...,Dawran ad Daydah,14.7393,44.2066,2,Yemen News Agency (SABA) - Houthi,National,"On 1 November 2024, protesters held a large Ho...",0,crowd size=large,1730758099
2,YEM93304,2024-11-01,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Yemen),Government of Yemen (2017-) Houthi,Protesters,...,Uzlat Al Aqibah,13.8786,43.7357,2,Yemen News Agency (SABA) - Houthi,National,"On 1 November 2024, protesters held a large Ho...",0,crowd size=large,1730758099
3,YEM93309,2024-11-01,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Yemen),Government of Yemen (2017-) Houthi,Protesters,...,Al Junayd,14.0842,44.0575,1,Yemen News Agency (SABA) - Houthi,National,"On 1 November 2024, protesters held a large Ho...",0,crowd size=large,1730758099
4,YEM93315,2024-11-01,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Yemen),Government of Yemen (2017-) Houthi,Protesters,...,Uzlat Al Ajum,14.0936,43.9495,2,Yemen News Agency (SABA) - Houthi,National,"On 1 November 2024, protesters held a large Ho...",0,crowd size=large,1730758099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476224,YEM32562,2015-01-01,2015,1,Political violence,Explosions/Remote violence,Remote explosive/landmine/IED,Unidentified Armed Group (Yemen),NaN,Political militia,...,Shibam,15.9214,48.6362,1,Barakish; Hadarem; Yemen Data Project,Local partner-National,Three soldiers were reportedly injured in an I...,0,NaN,1690843540
476225,YEM32563,2015-01-01,2015,1,Political violence,Battles,Armed clash,Military Forces of Yemen (2012-2022) Hadi,NaN,State forces,...,Nakhla,15.6160,45.1987,1,Al Jazeera; AP; Barakish; Yemen Data Project; ...,Local partner-National,"On 1 January 2015, Islah-affiliated tribesmen ...",9,NaN,1690843540
476226,YEM32564,2015-01-01,2015,1,Political violence,Battles,Armed clash,Military Forces of Yemen (2012-2022) Hadi,NaN,State forces,...,As Suhayl,15.5469,45.1265,1,Al Jazeera; AP; Barakish; Yemen Data Project; ...,Local partner-National,"On 1 January 2015, Islah-affiliated tribesmen ...",9,NaN,1690843540
476227,YEM32559,2015-01-01,2015,1,Political violence,Battles,Armed clash,AQAP: Al Qaeda in the Arabian Peninsula,Popular Resistance,Rebel group,...,Al Qayfa,14.5322,44.8228,2,Barakish; Yemen Data Project,Local partner-National,Houthi forces clashed with armed tribesmen bac...,0,NaN,1695070249


In [3]:
import keplergl
from keplergl import KeplerGl
import geopandas as gpd

# Data preparation - take random sample of 100k points (adjust n=500000 if your system can handle it)
df_map = df.dropna(subset=['latitude', 'longitude']).copy()


# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_map,
    geometry=gpd.points_from_xy(df_map.longitude, df_map.latitude)
)

# Calculate map center and zoom approximation
lat_center = (11 + 43) / 2
lon_center = (25 + 65) / 2

# Create map with optimized configuration
map_config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": lat_center,
            "longitude": lon_center,
            "zoom": 3.5,  # Adjusted to show the full area
            "dragRotate": False
        },
        "visState": {
            "layers": [{
                "type": "hexbin",
                "config": {
                    "dataId": "incidents",
                    "label": "Density Heatmap",
                    "colorRange": [
                        [255, 255, 204],  # Light yellow
                        [254, 217, 118],  # Orange
                    ],
                    "radius": 25,  # Hexagon size in pixels
                    "opacity": 0.8,
                    "coverage": 0.9,
                    "enable3d": False,
                    "sizeRange": [0, 100],
                    "coverageRange": [0, 500],
                    "elevationScale": 5,
                    "colorAggregation": "count",
                    "textLabel": [{
                        "field": None,
                        "color": [255, 255, 255],
                        "size": 18,
                        "offset": [0, 0],
                        "anchor": "start",
                        "alignment": "center"
                    }]
                }
            }],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "incidents": [{"name": "point_count", "format": None}]
                    },
                    "enabled": True
                }
            }
        }
    }
}

# Create map instance
map_ = KeplerGl(height=1500, config=map_config)

# Add data with proper type recognition
map_.add_data(data=gdf, name='incidents')

# Display in notebook
map_

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


C:\Users\macie\PycharmProjects\Data-visualisation\.venv\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 27.0, 'longitude': 45.0, 'zoom': 3.5, 'd…

In [5]:
from keplergl import KeplerGl
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import HTML, display

# Data preparation
df_map = df.dropna(subset=['latitude', 'longitude']).copy()

# Clean numerical data
df_map = df_map.replace([np.inf, -np.inf], np.nan)
df_map = df_map.dropna(subset=['latitude', 'longitude'])
df_map['latitude'] = pd.to_numeric(df_map['latitude'], errors='coerce')
df_map['longitude'] = pd.to_numeric(df_map['longitude'], errors='coerce')

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_map,
    geometry=gpd.points_from_xy(df_map.longitude, df_map.latitude)
)

# Map configuration (same as before)
map_config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 27,
            "longitude": 45,
            "zoom": 3.5,
            "dragRotate": False
        },
        "visState": {
            "layers": [{
                "type": "hexbin",
                "config": {
                    "dataId": "incidents",
                    "label": "Density Heatmap",
                    "colorRange": [
                        [255, 255, 204],
                        [254, 217, 118],
                    ],
                    "radius": 25,
                    "opacity": 0.8,
                    "coverage": 0.9,
                    "enable3d": False,
                    "colorAggregation": "count"
                }
            }]
        }
    }
}

# Create and display map
map_ = KeplerGl(height=600, config=map_config)
map_.add_data(data=gdf, name='incidents')

# Generate and display HTML properly
html_content = map_.save_to_html()
display(HTML(
    f'<div style="position: fixed; top: 0; left: 0; width: 100%; height: 100%; z-index: 9999;">'
    f'{html_content}'
    f'</div>'
))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


C:\Users\macie\PycharmProjects\Data-visualisation\.venv\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to keplergl_map.html!
